In [ ]:
! pip install --upgrade xarray zarr gcsfs cftime nc-time-axis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs


In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [ ]:
ds_processed=xr.open_dataset('/content/drive/MyDrive/CMIP6_pre-processed_data.nc')
source_list = ds_processed.model


<xarray.DataArray 'model' (model: 27)>
array(['CanESM5', 'AWI-CM-1-1-MR', 'EC-Earth3-Veg', 'CMCC-CM2-SR5', 'TaiESM1',
       'IPSL-CM6A-LR', 'NESM3', 'CAMS-CSM1-0', 'FGOALS-f3-L', 'GFDL-CM4',
       'CMCC-ESM2', 'MRI-ESM2-0', 'INM-CM5-0', 'BCC-CSM2-MR', 'INM-CM4-8',
       'EC-Earth3', 'IITM-ESM', 'CAS-ESM2-0', 'CESM2-WACCM', 'EC-Earth3-CC',
       'E3SM-1-1', 'GFDL-ESM4', 'MIROC6', 'FIO-ESM-2-0', 'EC-Earth3-Veg-LR',
       'KACE-1-0-G', 'NorESM2-MM'], dtype=object)
Coordinates:
  * model    (model) object 'CanESM5' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'

In [ ]:
gcs = gcsfs.GCSFileSystem(token='anon')

In [ ]:
def load_zarr_dset(df):
  # get the path to a specific zarr store (the first one from the dataframe above)
  zstore = df.zstore.values[-1]

  # create a mutable-mapping-style interface to the store
  mapper = gcs.get_mapper(zstore)

  # open it using xarray and zarr
  ds = xr.open_zarr(mapper, consolidated=True)
  return ds

In [ ]:
#time index to assign model times to
new_time_index=pd.date_range(start='1851-01-01',freq='M',periods=2988)
new_time_index

DatetimeIndex(['1851-01-31', '1851-02-28', '1851-03-31', '1851-04-30',
               '1851-05-31', '1851-06-30', '1851-07-31', '1851-08-31',
               '1851-09-30', '1851-10-31',
               ...
               '2099-03-31', '2099-04-30', '2099-05-31', '2099-06-30',
               '2099-07-31', '2099-08-31', '2099-09-30', '2099-10-31',
               '2099-11-30', '2099-12-31'],
              dtype='datetime64[ns]', length=2988, freq='M')

In [ ]:
ds_list = list()

for source in source_list:
    df_hist_tas=df.query  ("activity_id=='CMIP'         & table_id == 'Amon' & variable_id == 'tas' & experiment_id=='historical'& member_id=='r1i1p1f1' & source_id==@source")
    df_ssp585_tas=df.query("activity_id=='ScenarioMIP'  & table_id == 'Amon' & variable_id == 'tas' & experiment_id=='ssp585'    & member_id=='r1i1p1f1' & source_id==@source")

  #load
    ds_hist_tas   =load_zarr_dset(df_hist_tas)
    ds_ssp585_tas =load_zarr_dset(df_ssp585_tas)

  #concat
    ds_all = xr.concat([ds_hist_tas, ds_ssp585_tas], dim = ('time'), coords='all',compat='override')

  #global mean
    weights = np.cos(np.deg2rad(ds_all.tas.lat))

    ds_all = ds_all.weighted(weights).mean(dim=('lon', 'lat'))

  #slice to our specific period and reassign the new time index created above
    ds_all = ds_all.sel(time=slice('1851', '2099'))
    ds_all=ds_all.assign(time=new_time_index)

  #add to list
    ds_list.append(ds_all)

ob_all = xr.concat(ds_list, dim = 'model',coords='minimal',compat='override')

ob_all

<xarray.Dataset>
Dimensions:  (time: 2988, model: 27)
Coordinates:
    height   (time) float64 2.0 2.0 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0 2.0 2.0 2.0
  * time     (time) datetime64[ns] 1851-01-31 1851-02-28 ... 2099-12-31
Dimensions without coordinates: model
Data variables:
    tas      (model, time) float64 dask.array<chunksize=(1, 114), meta=np.ndarray>

In [ ]:
ob_all = ob_all.assign(model = source_list)
ob_all

<xarray.Dataset>
Dimensions:  (model: 27, time: 2988)
Coordinates:
  * model    (model) object 'CanESM5' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
  * time     (time) datetime64[ns] 1851-01-31 1851-02-28 ... 2099-12-31
Data variables:
    tas      (model, time) float64 dask.array<chunksize=(1, 114), meta=np.ndarray>

In [ ]:
ob_all.to_netcdf("global_data_processed.nc", mode = 'w', format = "NETCDF4", engine = 'netcdf4')